In [1]:
i0=0
i1=0
i2=0

In [2]:
import pandas as pd
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
from tqdm import tqdm
import sys
sys.path.append('/home/icb/manuel.gander/Atl/notebooks/')
import utils
utils = importlib.reload(utils)
import scipy
import pickle

import warnings
from scipy.stats import ConstantInputWarning
from sklearn.exceptions import ConvergenceWarning

# Ignore ConstantInputWarning
warnings.filterwarnings("ignore", category=ConstantInputWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0]
l1_ratios = [0.01, 0.03, 0.1, 0.3, 0.5, 0.9, 0.99]
kn=10**9
reps=5
methods = ['auc', 'ic50']

alpha = alphas[i0]
l1_ratio = l1_ratios[i1]
method = methods[i2]

source = 'GDSC2'
dataset = 'RNA'


D_prot, features, celllines = utils.load_dataset(dataset)
dfv = utils.prep_viability_AUCs2(D_prot)
dfv = dfv[dfv['Source']==source].copy()
dfv = dfv[np.isfinite(dfv['ic50'])].copy()
dfv = dfv[np.isfinite(dfv['AUC'])].copy()

dfv.loc[dfv['ic50']>dfv['max_dose']/10, 'ic50']=dfv[dfv['ic50']>dfv['max_dose']/10]['max_dose']/10
dfv.loc[dfv['ic50']<dfv['min_dose']/10, 'ic50']=dfv[dfv['ic50']<dfv['min_dose']/10]['min_dose']/10


if method=='ic50':
    dfv['AUC'] = dfv['ic50'].copy()
    

with open('/home/icb/manuel.gander/Atl/data/common_drugs.pkl', 'rb') as file:
    Dd = pickle.load(file)
drugs = Dd[source]

100%|███████████████████████████████████████████████████████████████████████| 1960464/1960464 [00:01<00:00, 1398821.27it/s]


In [2]:
frames=[]
for dr in tqdm(drugs):
    dfvs=dfv[dfv['PubChem_CID']==dr].copy()

    ccls=sorted(set(dfvs['Cello']))
    res={}
    res['pearsons']=[]
    res['spearmans']=[]
    res['l1_ratios']=[]
    res['RMSE']=[]
    res['RMSE_mean_model']=[]
    res['td']=[]
    if len(ccls)>30:
        D_holdout = utils.get_holdouts_balanced(dfvs, reps, source0='all')
        if len(D_holdout)>0:
            for hs in range(reps):
                holdouts=D_holdout[hs]
                dfv0, dfh=utils.split_of_validation_cell_lines(dfvs, ccls=holdouts)
                ma = dfv0['AUC'].mean()
                sa = dfv0['AUC'].std()
                dfv0['AUC'] = (dfv0['AUC']-ma)/sa
                dfh['AUC'] = (dfh['AUC']-ma)/sa
                
                train_input=utils.dfv_to_train_arrays(dfv0, D_prot)
                mean = train_input[1].mean()

                # Preselection of kn features and inference
                t0=time.time()
                predictions, ys = utils.calc_predictions_combinations(dfv0, dfh, D_prot, kn, alpha, l1_ratio)
                td = time.time()-t0
                p=predictions[0]
                y=ys[0]

                res['pearsons'].append(scipy.stats.pearsonr(p, y).statistic)
                res['spearmans'].append(scipy.stats.spearmanr(p, y).statistic)
                res['RMSE'].append(np.sqrt(np.mean((p-y)**2)))
                res['l1_ratios'].append(np.mean(abs(p-y))/np.mean(abs(mean-y)))
                res['RMSE_mean_model'].append(np.sqrt(np.mean((mean-y)**2)))
                res['td'].append(td)

    df = pd.DataFrame(res)
    df['PubChem_CID']=dr
    df['ind']=range(len(df))
    df['n_ccls'] = len(dfvs)
    frames.append(df)
Df = pd.concat(frames, ignore_index=True)
Df['source']=source
Df['dataset']=dataset
Df['alpha']=alpha
Df['l1_ratio']=l1_ratio
Df['kn'] = kn
Df['method'] = method

  2%|█▎                                                                                   | 1/66 [01:21<1:28:18, 81.52s/it]


In [4]:
df

,pearsons,spearmans,l1_ratios,RMSE,RMSE_mean_model,td,PubChem_CID,ind,n_ccls
0,0.619031,0.691554,0.775164,0.880683,1.109358,10.408731,11707110,0,611
1,0.659933,0.679752,0.780104,0.712240,0.926072,9.555134,11707110,1,611
2,0.681065,0.602244,0.687418,0.713013,0.972693,9.619290,11707110,2,611
3,0.736113,0.712398,0.700665,0.748615,1.084208,9.659278,11707110,3,611
4,0.560642,0.549374,0.913477,0.762677,0.854207,9.550618,11707110,4,611


In [ ]:
Df.to_pickle(f'/home/icb/manuel.gander/Atl/data/dones_grid/{alpha}_{l1_ratio}_{method}.pkl')